[![View notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/tree/main/tutorials/collisions/collisions_playground.ipynb)   
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/tutorials/collisions/collisions_playground.ipynb)   
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/tutorials/collisions/collisions_playground.ipynb)

## Cloud Microphysics: Part 2
- Collisions and coalescence of cloud droplets

Based on Fig. 2 from Shima et al. 2009 (Q. J. R. Meteorol. Soc. 135)  "_The super‐droplet method for the numerical simulation of clouds and precipitation: a particle‐based and probabilistic microphysics model coupled with a non‐hydrostatic model_."   
https://doi.org/10.1002/qj.441

## Collision/coalescence 

The process of droplets colliding and coalescence, together refered to as collection, is the mechanism by which cloud droplets grow and eventually grow large enough to precipitate.
The collection process depends on these two processes, first two droplets colliding, and second that collision resulting in the coalescence of a new larger droplet.

In models we parameterize this collection process stochastically by solving what is known as the SCE: Stochastic Collection Equation.
And we write the probability that two droplets collide (collision rate) in terms of a "kernel": $K(x,y)$, where $x$ and $y$ are the sizes of the two droplets.

In this example, we consider the most basic kernel called the Golovin kernel, which is a linear kernel of the form $K(x,y) = b(x+y)$.

Below is a drawing from Lamb and Verlinde's "_The Physics and Chemistry of Clouds_" illustrating the geometry of droplet collisions.

<img src="./collection_droplet.svg" width="500">

## PySDM box model widget

In this homework assignment, and with this `PySDM` example notebook, you have the chance to explore how particle size, collision kernel, and spectral resolution (number of superdroplets used to represent the droplet size distribution) influence the growth of a population of cloud droplets as they undergo collision and coalescence. 

In this box setup, we can focus on only the collision/coalescence process while ignoring the hygroscopic growth of particles and activation of aerosols considered in Part 1, as well as fluid flow and mixing from a 2D or 3D simulation.

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install "open_atmos_jupyter_utils"
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [ ]:
from numpy import errstate
import os

from PySDM import Formulae
from PySDM.dynamics.collisions.collision_kernels import Golovin
from PySDM.initialisation import spectra
from PySDM.physics import si

from PySDM_examples.utils import widgets

from PySDM_examples.Shima_et_al_2009.tutorial_plotter import SpectrumPlotter
from PySDM_examples.Shima_et_al_2009.tutorial_settings import Settings
from PySDM_examples.Shima_et_al_2009.tutorial_example import run

In [ ]:
def demo(*, _freezer, _n, _b, _r, _smooth):
    frm = Formulae()
    with _freezer:
        with errstate(all='raise'):
            n_step = 3600
            n_plot = 3
            settings = Settings(steps=[i * (n_step // n_plot) for i in range(n_plot + 1)])
            settings.n_sd = 2 ** _n
            settings.adaptive = True
            settings.dt = 10
            settings.kernel = Golovin(b=_b / si.second)
            settings.X0 = frm.trivia.volume(radius=_r * si.micrometres)
            settings.spectrum = spectra.Exponential(
                norm_factor=settings.norm_factor, scale=settings.X0
                )
            states, _ = run(settings, (widgets.ProgbarUpdater(progbar, settings.output_steps[-1]),))

        with errstate(invalid='ignore'):
            plotter = SpectrumPlotter(settings)
            plotter.smooth = _smooth
            for step, state in states.items():
                plotter.plot(state, step * settings.dt)
            plotter.show()

## Widget

Play around with the widget and change the number of superdroplets ($n_{SD} = 2^X$), slope parameter ($b$) in the Golovin collision kernel, and the scale parameter in the droplet size distribution ($r$).

<div class="alert alert-block alert-info">
<b>Note:</b> Running the box model takes a few seconds, so be patient after you move one of the sliders.</div>

The plot generated shows the evolution of the droplet size distribution at various time points (in color). Plotted underneath (in black) is the analytical solution, which exists for this simple Golovin collision kernel.

In [ ]:
style = {'description_width': 'initial'}
n_SD = widgets.IntSlider(value=12, min=6, max=18, step=1, 
                         description='log2(nSD)', continuous_update=False, style=style)
b = widgets.FloatSlider(value=1.5e3, min=1e3, max=2e3, step=1e2, 
                        description='b (s-1)', continuous_update=False,
                        readout_format='.1e', style=style)
r = widgets.IntSlider(value=30, min=25, max=35, step=1, 
                        description='r (um)', continuous_update=False, style=style)
sliders = widgets.HBox([n_SD, b, r])

smooth = widgets.Checkbox(value=True, description='smooth plot')
options = [smooth]
boxes = widgets.HBox(options)
freezer = widgets.Freezer([n_SD, b, r])
inputs = {'_freezer': freezer, '_n': n_SD, '_b': b, '_r': r, '_smooth': smooth}
progbar = widgets.IntProgress(min=0, max=100, description='%')

if 'CI' not in os.environ:
    widgets.display(sliders, boxes, progbar, widgets.interactive_output(demo, inputs))

## Questions

1. How does the shape of the droplet size distribution change over time as particles collide and coalesce?

2. For the Golovin collision kernel there is an analytical solution, plotted in the black curve. 
How many superdroplets are needed to robustly simulate droplet collection?

3. What does the `b` parameter in the collision kernel control?

4. How does the mean radius of the droplets affect the collision rate? 